## Parse patient zero results - proximity

In [1]:
import numpy as np
import numba as nb
import pandas as pd

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../../src')
import networkx as nx

In [2]:
import io_m.libsaving as libsaving
import io_m.io_utils as io_utils

from importlib import reload

In [3]:
from epigen import epidemy_gen

In [4]:
from epigen.base import EpInstance

In [5]:
type_graph="i_bird"
N=10
d=3
height=3
lam=0.5
mu=1e-3
t_limit=12
scale=2
#gamma=1e-3
#path_contacts="../patient_zero/work/work_13_contacts.npz"
#small_lambda_limit=0

gamma=8e-5
small_lambda_limit=300
path_contacts="work_13_contacts.npz"
path_contacts="i_bird_contacts.npz"

#path_contacts="i_bird_contacts.npz"

p_edge=1
nsrc=1


In [6]:
seeds=range(1,2)

In [7]:
INSTANCES = {s:EpInstance(type_graph, N, d, t_limit, 
                                    lam, mu, s, float(p_edge), 
                                    n_source=nsrc) for s in seeds}

In [8]:
str(INSTANCES[1])

'i_bird_n_10_d_3_tlim_12_lam_0.5_mu_0.001_s_1_pe_1.0'

In [9]:
extra_gen = {"h":height,
            "scale":scale}

extra_gen.update({"gamma":gamma, 
                  "small_lambda_limit":small_lambda_limit, 
                  "path_contacts": path_contacts})

In [10]:
start_conf=0
num_conf=100

In [11]:
reload(epidemy_gen)
sources=set([50])
DATA_ALL = {s: 
            epidemy_gen.epidemy_gen_epinstance(inst, 
                                               lim_infected=1, num_conf=num_conf,
                                               extra_gen=extra_gen, 
                                               verbose=False,
                                              num_sources=nsrc,
                                              sources=sources) 
            for s, inst in INSTANCES.items()}


number of contacts: 14010
Lim infected: 1, Lim max infected: 327
Num sources:  1
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources 

In [13]:
from annfore.net import nn_sir_path_obs
from annfore.utils.graph import find_neighs
from annfore.models import common as en_common
from annfore.models import sir_model_N_obs
from annfore.learn.losses import loss_fn_coeff, loss_fn_coeff_p_sus
from annfore.learn.train import train_beta, train_saving_marginals, make_training_step_local
from annfore.learn.opt import make_opt

from annfore.learn.train_params import opt_param_init, train_beta_params
from annfore.learn.train_params import learn_lamb_mu, learn_gamma_mu
from annfore.learn.l_utils import make_beta_sequence_three as make_beta_seq


In [17]:
DATA_ALL.keys()

dict_keys([1])

In [20]:
func_layers = [1, 1]
contacts = DATA_ALL[1]["contacts"]
N = int(max(contacts[:, 1]) + 1)
mu = 0
p_source = 1/N
neighs = find_neighs(contacts,N=N,only_minor=True, next_near_neigh=True)

In [25]:
instance_num = 0
last_obs = DATA_ALL[1]["test"][instance_num][1]
obs_list_net = []
for i, s in enumerate(last_obs):
    obs_list_net.append([i,s,t_limit])

In [30]:
device = "cpu"
my_net = nn_sir_path_obs.SIRPathColdObs(neighs,
                    t_limit+1,
                    obs_list=obs_list_net,
                    hidden_layer_spec=func_layers,
                    device = device,
                    bias=True,
                    )

[0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [33]:
p_mc = np.exp(np.log(p_source) - t_limit)
p_obs = p_mc
model = sir_model_N_obs.SirModel(contacts,
                        mu = mu,
                        device = device,
                            p_source=p_source,
                            p_obs=p_obs,
                            p_w=p_mc)
model.set_obs(obs_list)

In [37]:
num_samples = 1000
lr=1e-3
optimizer = []
for i in range(N):
    if len(my_net.params_i[i]):
        optimizer.append(make_opt(my_net.params_i[i], lr=lr))
    else:
        optimizer.append([])
t_obs = t_limit
n_beta_steps = 10000
betas = np.linspace(0,1,n_beta_steps+1)[:-1]
name_file_instance = "i_bird" + "_" + str(instance_num)


In [39]:
loss_fun = loss_fn_coeff
extra_args = None

results = train_beta(my_net, optimizer,
                        model, name_file_instance,
                        loss_fun, t_obs,
                        num_samples=num_samples,
                        train_step = make_training_step_local,
                        betas=betas, save_every=200,
                        loss_extra_args=extra_args)